In [6]:
# download video from youtube or a given public URL
url = "https://www.youtube.com/watch?v=JmNCPK15Bc0"
import youtube_dl
ydl_opts = {'outtmpl': 'input.mp4'}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] JmNCPK15Bc0: Downloading webpage
[youtube] JmNCPK15Bc0: Downloading video info webpage
[download] input.mp4 has already been downloaded
[download] 100% of 13.37MiB


In [7]:
# check if the video got downloaded correctly
from IPython.display import HTML
HTML('<iframe width=640 height=480 src=input.mp4>')

In [8]:
# load VGG-16 pre-trained model in Keras
import keras
from keras.applications.vgg16 import VGG16
model = VGG16()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# import few other libraries to pre-process image in Keras
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [10]:
import numpy as np

In [11]:
# required to process the model output of keras
from keras.applications.vgg16 import decode_predictions

In [14]:
# load video and process it frame by frame
import cv2
stream = cv2.VideoCapture('input.mp4')

# for Mac OS, this is the code for writing mp4 files
codec = cv2.VideoWriter_fourcc('m','p','4','v')
frame_rate = 10
# save tagged video output in output.mp4 file
output = cv2.VideoWriter('output.mp4', codec, frame_rate, (1280,720), True)

while 1:
    flag, frame = stream.read()
    if flag:
        # resize frame for keras vgg-16 model
        bgr_frame = cv2.resize(frame, (224, 224))
        # covert OpenCV's BGR format to RGB for Keras
        rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

        # following few routine steps to pre-process input image for Keras
        input_image = image.img_to_array(rgb_frame)
        input_image = np.expand_dims(input_image, axis=0)
        input_image = preprocess_input(input_image)
        
        # finally, ready to run the model
        predictions = model.predict(input_image)

        text = ""
        # decoding the model output
        for label in decode_predictions(predictions, top=5)[0]:
            # apply some threshold on confidence scores
            if label[2] > 0.5:
                text = label[1]
                break

        # print text on the image frame
        cv2.putText(frame, text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0,255,255))
        output.write(frame)
        cv2.imshow('Wild Life', frame)
        
        # standard stuff to wait after each frame
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
output.release()
stream.release()
cv2.destroyAllWindows()

In [15]:
# lets check the output file created
HTML('<iframe width=640 height=480 src=output.mp4>')

In [16]:
# yay, done...